In [1]:
import pandas as pd
import nltk.stem as ns
from pickle import dump, load
from nltk.corpus import stopwords
import nltk
import re

import rich.progress

with rich.progress.open('/storage10/shanleng/tmp/metadata.csv', 'rb') as file:
    df = pd.read_csv(file)
    
print(df.shape)
df.head(3)

Output()

/tmp/ipykernel_2118726/1372825044.py:11: DtypeWarning: Columns (1,4,5,6,13,14,15,16) have mixed types. Specify 
dtype option on import or set low_memory=False.
  df = pd.read_csv(file)

(1056660, 19)


,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN


In [2]:
df.isnull().any(axis = 0)

cord_uid            False
sha                  True
source_x            False
title                True
doi                  True
pmcid                True
pubmed_id            True
license             False
abstract             True
publish_time         True
authors              True
journal              True
mag_id               True
who_covidence_id     True
arxiv_id             True
pdf_json_files       True
pmc_json_files       True
url                  True
s2_id                True
dtype: bool

In [2]:
df_ = df[['cord_uid', 'abstract', 'title', 'pdf_json_files', 'pmc_json_files']]
df_ = df_[~((df_['pdf_json_files'].isnull()) & (df_['pmc_json_files'].isnull()))]
df_.shape

(389899, 5)

In [20]:
with open('/afs/cs.wisc.edu/u/l/e/leng/private/STAT605/COVID/valid_metadata','wb') as filepath: 
    dump(df_, filepath)

In [21]:
df_.isnull().any(axis = 0)

cord_uid          False
abstract           True
title              True
pdf_json_files     True
pmc_json_files     True
dtype: bool

In [3]:
titles = df_[['title']].dropna(how = 'any', axis = 0).title.tolist()
len(titles)

389895

In [4]:
abstracts = df_[['abstract']].dropna(how = 'any', axis = 0).abstract.tolist()
len(abstracts)

334585

In [5]:
stop = stopwords.words('english')

filter_text = re.sub('[{}]'.format('!"#$%&()*+,./:;<=>?@[\\]^_`{|}~'),'',titles[0]).replace('\n', '')
filter_text = [word.lower() for word in filter_text.split(' ') if word not in stop and not word.isdigit()]
filter_text

['clinical',
 'features',
 'culture-proven',
 'mycoplasma',
 'pneumoniae',
 'infections',
 'king',
 'abdulaziz',
 'university',
 'hospital',
 'jeddah',
 'saudi',
 'arabia']

In [6]:
titles_ = []

for t in df_.fillna(0).title.tolist():
    if t == 0:
        titles_.append(t)
    else:
        filter_text = re.sub('[{}]'.format('!"#$%&()*+,./:;<=>?@[\\]^_`{|}~'),'',t).replace('\n', '')
        filter_text = [word.lower() for word in filter_text.split(' ') if word not in stop and not word.isdigit()]
        titles_.append(filter_text)

In [7]:
titles_[6]

['debate',
 'transfusing',
 'normal',
 'haemoglobin',
 'levels',
 'improve',
 'outcome']

In [7]:
abstracts_ = []
count = 0

for a in df_.fillna(0).abstract.tolist():
    if a == 0:
        abstracts_.append(a)
    else:
        filter_text = re.sub('[{}]'.format('!"#$%&()*+,./:;<=>?@[\\]^_`{|}~'),'',a).replace('\n', '')
        filter_text = [word.lower() for word in filter_text.split(' ') if word not in stop and not word.isdigit()]
        abstracts_.append(filter_text)
        
    count += 1
    if count%50000 == 0:
        print(count, end = ' ')

50000 100000 150000 200000 250000 300000 350000 

In [9]:
abstracts_[0]

['objective',
 'this',
 'retrospective',
 'chart',
 'review',
 'describes',
 'epidemiology',
 'clinical',
 'features',
 'patients',
 'culture-proven',
 'mycoplasma',
 'pneumoniae',
 'infections',
 'king',
 'abdulaziz',
 'university',
 'hospital',
 'jeddah',
 'saudi',
 'arabia',
 'methods',
 'patients',
 'positive',
 'm',
 'pneumoniae',
 'cultures',
 'respiratory',
 'specimens',
 'january',
 'december',
 'identified',
 'microbiology',
 'records',
 'charts',
 'patients',
 'reviewed',
 'results',
 'patients',
 'identified',
 'required',
 'admission',
 'most',
 'infections',
 'community-acquired',
 'the',
 'infection',
 'affected',
 'age',
 'groups',
 'common',
 'infants',
 'pre-school',
 'children',
 'it',
 'occurred',
 'year-round',
 'common',
 'fall',
 'spring',
 'more',
 'three-quarters',
 'patients',
 'comorbidities',
 'twenty-four',
 'isolates',
 'associated',
 'pneumonia',
 'upper',
 'respiratory',
 'tract',
 'infections',
 'bronchiolitis',
 'cough',
 'fever',
 'malaise',
 'common',

In [8]:
verb = ['VBD', 'VB', 'VBG', 'VBN', 'VBZ', 'VBG']
noun = ['NN', 'NNS', 'NNP', 'NNPS']
adj = ['JJ', 'JJR', 'JJS']
adv = ['RB', 'RBR', 'RBS', 'UH']

abs_n = []
abs_v = []
abs_aj = []
abs_av = []

count = 0

lemmatizer = ns.WordNetLemmatizer()

for a in abstracts_:
    if a == 0:
        v_ = 0
        n_ = 0
        adj_ = 0
        adv_ = 0
    else:
        a = list(filter(lambda s: len(s) > 1,a))
        tag = [nltk.pos_tag([w])[0] for w in a]
        v_ = [tag[i][0] for i in range(len(tag)) if tag[i][1] in verb]
        n_ = [tag[i][0] for i in range(len(tag)) if tag[i][1] in noun]
        adj_ =  [tag[i][0] for i in range(len(tag)) if tag[i][1] in adj]
        adv_ =  [tag[i][0] for i in range(len(tag)) if tag[i][1] in adv]
        
        v_ = [lemmatizer.lemmatize(v, pos='v') for v in v_]
        n_ = [lemmatizer.lemmatize(n, pos='n') for n in n_]
        adj_ = [lemmatizer.lemmatize(aj, pos='a') for aj in adj_]
        adv_ = [lemmatizer.lemmatize(av, pos='r') for av in adv_]
    
    abs_n.append(n_)
    abs_v.append(v_)
    abs_aj.append(adj_)
    abs_av.append(adv_)
    
    count += 1
    if count%2000 == 0:
        print(count, end = ' ')

2000 4000 6000 8000 10000 

KeyboardInterrupt: 

In [12]:
abs_av[6]

['critically', 'previously', 'clinically', 'critically']

In [2]:
meta_words = load(open('/afs/cs.wisc.edu/u/l/e/leng/private/STAT605/COVID/meta_words', 'rb'))
meta_words.head(3)

,cord_uid,title,abs_n,abs_v,abs_aj,abs_av
0,ug7v899j,"[clinical, features, culture-proven, mycoplasm...","[objective, retrospective, chart, review, desc...","[identify, review, identify, require, occur, a...","[clinical, positive, most, community-acquired,...",[more]
1,02tnwd4m,"[nitric, oxide, pro-inflammatory, mediator, lu...","[inflammatory, disease, respiratory, tract, pr...","[associate, elevate, increase, know, presume, ...","[nitric, oxidative, anti-microbial, anti-oxida...","[commonly, often, comprehensively, instead]"
2,ejv2xln0,"[surfactant, protein-d, pulmonary, host, defense]","[surfactant, protein-d, sp-d, participates, in...","[inhale, synthesize, secrete, express, line, e...","[organic, epithelial, epithelial, various, gen...","[also, also, specifically, abnormally, appropr..."
